---

## Lección 2: Probabilidad y Estadística

### 1. Definición de Eventos Aleatorios y Árbol de Probabilidad

Para modelar el comportamiento de los estudiantes, debemos entender que cada medición es un experimento aleatorio. Definimos los eventos principales basados en los criterios de riesgo establecidos en la etapa anterior.

#### A. Definición de Eventos
Definimos tres eventos dicotómicos (cumple / no cumple) para cada estudiante seleccionado al azar:

* **Evento $D$ (Descanso):** El estudiante duerme $\geq 7$ horas diarias.
* **Evento $E$ (Ejercicio):** El estudiante realiza $\geq 150$ minutos de actividad física semanal.
* **Evento $A$ (Alimentación):** El estudiante consume ultraprocesados $\leq 3$ veces por semana.

Los complementos de estos eventos ($D^c, E^c, A^c$) representan las situaciones de riesgo para el Área de Salud.

#### B. Espacio Muestral ($\\Omega$)
El espacio muestral $\\Omega$ es el conjunto de todos los perfiles posibles que puede tener un estudiante respecto a estos tres hábitos. Al tener 3 variables con 2 posibilidades cada una, tenemos $2^3 = 8$ resultados posibles:

$$\Omega = \{ (DEA), (DEA^c), (DE^cA), (DE^cA^c), (D^cEA), (D^cEA^c), (D^cE^cA), (D^cE^cA^c) \}$$



#### C. Árbol de Probabilidad
El árbol de probabilidad permite visualizar la probabilidad conjunta de cada perfil. Aunque para el análisis inicial asumiremos **independencia** entre eventos (el hecho de dormir bien no garantiza que el alumno haga ejercicio), en la realidad estos eventos suelen estar correlacionados.

**Representación lógica del árbol:**
1.  **Nivel 1 (Sueño):** Probabilidad de dormir bien $P(D)$ vs. Déficit de sueño $P(D^c)$.
2.  **Nivel 2 (Ejercicio):** Para cada rama anterior, se divide en Activo $P(E)$ vs. Sedentario $P(E^c)$.
3.  **Nivel 3 (Alimentación):** Para cada combinación, se divide en Saludable $P(A)$ vs. No Saludable $P(A^c)$.



### 2. Plan de Muestreo: Muestreo Aleatorio Estratificado

Para garantizar que la inferencia sea válida y representativa de toda la diversidad universitaria, se ha seleccionado el **Muestreo Aleatorio Estratificado**. Aunque en la fase inicial se consideró el muestreo aleatorio simple, la estratificación permite reducir la varianza global y asegurar que grupos minoritarios (como facultades con menos alumnos) tengan una representación proporcional en los datos.

#### A. Definición de Estratos
La población se dividirá en estratos basados en la **Facultad de pertenencia**, ya que se asume que la carga académica y los horarios varían significativamente entre áreas (ej. Salud vs. Ingeniería), influyendo directamente en los hábitos de sueño y ejercicio.

#### B. Mecanismo de Afijación Proporcional
Se utilizará una **afijación proporcional**, donde el tamaño de la muestra en cada estrato es proporcional al tamaño del estrato en la población total. 

Si nuestra muestra objetivo es $n = 385$:
1. Se obtiene el padrón total de alumnos por facultad.
2. Se calcula el peso relativo de cada facultad ($W_i$).
3. Se selecciona aleatoriamente a los estudiantes dentro de cada facultad hasta completar su cuota.



> **Nota sobre la simulación:** Las proporciones por facultad utilizadas en el código (Ingeniería 40%, Salud 30%, Artes y Humanidades 30%) son valores hipotéticos asumidos para ilustrar el método de afijación proporcional. En la implementación real, estos pesos se obtendrán del padrón oficial de estudiantes activos.

#### C. Justificación Técnica
* **Reducción del Error:** Al segmentar por facultad, disminuimos la heterogeneidad interna de cada grupo, lo que resulta en estimaciones de la media ($\mu$) más precisas.
* **Representatividad:** Evitamos el riesgo de que el azar concentre la encuesta en una sola carrera, lo que sesgaría los resultados hacia un solo tipo de perfil de estudiante.
* **Marco Muestral:** Se utilizará la base de datos de correos institucionales como marco muestral, asignando un número único a cada estudiante y seleccionándolos mediante un generador de números aleatorios por cada estrato.

---

#### D. Reglas de Probabilidad Aplicadas

Para el análisis de los datos recolectados, se aplicarán las siguientes leyes probabilísticas:

1.  **Regla de la Suma (Probabilidad de Riesgo):** Para calcular la probabilidad de que un estudiante tenga *al menos uno* de los hábitos de riesgo ($P(D^c \cup E^c \cup A^c)$).
2.  **Probabilidad Condicional:** Fundamental para responder preguntas de gestión: *"¿Cuál es la probabilidad de que un alumno sea sedentario, dado que pertenece a la facultad de medicina?"* ($P(E^c | \text{Salud})$).
3.  **Teorema de Bayes:** Si detectamos a un estudiante con fatiga crónica (síntoma), ¿cuál es la probabilidad de que su causa principal sea el déficit de sueño ($D^c$)?



### 3. Simulación del Diseño Muestral

Para llevar a la práctica el plan de muestreo estratificado, se generará un conjunto de datos simulado con **n = 385 registros**, tal como fue definido en el diseño del estudio (Lección 1, Sección 5B), con un nivel de confianza del 95% y un error muestral del 5%. Esta simulación respetará la proporción de estudiantes por facultad y asignará valores a las variables de salud utilizando las distribuciones de probabilidad teóricas (Normal y Poisson).

In [9]:
import pandas as pd
import numpy as np

# Configurar semilla para asegurar la reproducibilidad de los datos
np.random.seed(42)

# 1. Definir el tamaño de la muestra
n_total = 385
proporciones = {'Ingeniería': 0.40, 'Salud': 0.30, 'Artes y Humanidades': 0.30}

estratos = []
# Convertimos a lista para manejar el último elemento por separado
llaves = list(proporciones.keys())

for i in range(len(llaves)):
    facultad = llaves[i]
    if i == len(llaves) - 1:
        # Para la última facultad (Artes), simplemente completamos lo que falte
        cantidad = n_total - len(estratos)
    else:
        # Para las demás, usamos la proporción
        cantidad = int(n_total * proporciones[facultad])
    
    estratos.extend([facultad] * cantidad)

# Ahora 'estratos' tendrá exactamente 385 elementos y el DataFrame funcionará.
# 2. Simular las variables cuantitativas según sus distribuciones
# Horas de sueño: Distribución Normal (media=6.5h, desviación=1.2h)
horas_sueno = np.random.normal(loc=6.5, scale=1.2, size=n_total)
horas_sueno = np.clip(horas_sueno, 3, 11) # Acotar a valores biológicamente posibles

# Minutos de ejercicio: Distribución Normal (media=140m, desviación=45m)
min_ejercicio = np.random.normal(loc=140, scale=45, size=n_total)
min_ejercicio = np.clip(min_ejercicio, 0, 500) # Evitar valores negativos

# Consumo de ultraprocesados: Distribución de Poisson (lambda=4 veces por semana)
veces_ultraprocesados = np.random.poisson(lam=4, size=n_total)

# 3. Integrar los datos en un DataFrame
df_muestra = pd.DataFrame({
    'ID_Estudiante': range(1, n_total + 1),
    'Facultad': estratos,
    'Horas_Sueno': np.round(horas_sueno, 1),
    'Minutos_Ejercicio': np.round(min_ejercicio, 0).astype(int),
    'Veces_Ultraprocesados': veces_ultraprocesados
})

# Desordenar las filas para simular la recolección aleatoria
df_muestra = df_muestra.sample(frac=1).reset_index(drop=True)

# Visualizar la cabecera del conjunto de datos
df_muestra.head()

df_muestra.to_csv('Muestra_universidad.csv', index=False)

### 4. Calcular probabilidades básicas de eventos

Una vez definida la muestra simulada, utilizamos los axiomas de probabilidad para cuantificar el riesgo en la población estudiantil. Para esto, definimos nuestros eventos de éxito (saludables) basándonos en los datos recolectados:

* **Evento $D$:** El estudiante duerme 7 horas o más.
* **Evento $E$:** El estudiante realiza 150 minutos o más de ejercicio a la semana.
* **Evento $A$:** El estudiante consume ultraprocesados 3 veces o menos a la semana.

A continuación, calculamos las probabilidades básicas (Empíricas) usando nuestra muestra simulada de 385 registros.



#### A. Eventos Complementarios
El complemento representa la probabilidad de que ocurra lo contrario a nuestro evento de interés (es decir, el escenario de riesgo). 
Si $P(D)$ es la probabilidad de dormir bien, el déficit de sueño es su complemento $P(D^c)$:

$$P(D^c) = 1 - P(D)$$

#### B. Intersección de Eventos (Y)
Representa la probabilidad de que dos eventos ocurran al mismo tiempo. Por ejemplo, un estudiante que tiene hábitos óptimos de descanso **y** de actividad física al mismo tiempo ($D \cap E$).

$$P(D \cap E) = \frac{\text{Casos favorables que cumplen D y E}}{\text{Total de la muestra}}$$

#### C. Unión de Eventos (O)
Representa la probabilidad de que ocurra al menos uno de los eventos. Nos sirve para medir el riesgo general. Por ejemplo, la probabilidad de que un estudiante tenga déficit de sueño **o** sea sedentario ($D^c \cup E^c$). 
Por la regla de la suma, debemos restar la intersección para no contar dos veces a los que tienen ambos problemas:

$$P(D^c \cup E^c) = P(D^c) + P(E^c) - P(D^c \cap E^c)$$

In [10]:
# Definimos las condiciones de los eventos (Filtros booleanos)
evento_D = df_muestra['Horas_Sueno'] >= 7
evento_E = df_muestra['Minutos_Ejercicio'] >= 150

# A. Probabilidad y Complementario (Déficit de sueño)
prob_D = evento_D.mean()
prob_D_comp = 1 - prob_D  # Equivalente a (df_muestra['Horas_Sueno'] < 7).mean()

print(f"P(D) - Probabilidad de dormir >= 7h: {prob_D:.2f}")
print(f"P(D^c) - Probabilidad de déficit de sueño: {prob_D_comp:.2f}")

# B. Intersección (Descansa bien Y hace ejercicio)
prob_D_int_E = (evento_D & evento_E).mean()
print(f"P(D ∩ E) - Duerme bien Y hace ejercicio: {prob_D_int_E:.2f}")

# C. Unión de riesgos (Déficit de sueño O Sedentarismo)
evento_D_comp = ~evento_D
evento_E_comp = ~evento_E

prob_riesgo_union = evento_D_comp.mean() + evento_E_comp.mean() - (evento_D_comp & evento_E_comp).mean()
print(f"P(D^c U E^c) - Riesgo de mal sueño O sedentarismo: {prob_riesgo_union:.2f}")

# D. Probabilidad del Evento A (Alimentación saludable)
evento_A = df_muestra['Veces_Ultraprocesados'] <= 3
prob_A = evento_A.mean()
prob_A_comp = 1 - prob_A
print(f"P(A) - Probabilidad de alimentación saludable (<=3 veces/sem): {prob_A:.2f}")
print(f"P(A^c) - Proporción con consumo frecuente de ultraprocesados (>3 veces/sem): {prob_A_comp:.2f}")

P(D) - Probabilidad de dormir >= 7h: 0.34
P(D^c) - Probabilidad de déficit de sueño: 0.66
P(D ∩ E) - Duerme bien Y hace ejercicio: 0.14
P(D^c U E^c) - Riesgo de mal sueño O sedentarismo: 0.86
P(A) - Probabilidad de alimentación saludable (<=3 veces/sem): 0.43
P(A^c) - Proporción con consumo frecuente de ultraprocesados (>3 veces/sem): 0.57
